wikipedia retriever

In [19]:
from langchain_community.retrievers import WikipediaRetriever

retriever=WikipediaRetriever(wiki_client=any,top_k_results=2,lang='en')

query1 = "the geopolitical history of india and pakistan from the perspective of a american"

# Get relevant Wikipedia documents
docs = retriever.invoke(query1)

docs

[Document(metadata={'title': 'Pakistan–United States relations', 'summary': 'Pakistan and the United States established relations on 15 August 1947, a day after the independence of Pakistan, when the United States became one of the first nations to recognise the country.\nThe relationship between the two nations has been described as a "roller coaster" characterised by close coordination and lows marked by deep bilateral estrangement. Despite its troubled history, the Pakistani military once occupied an important place in American geopolitical strategy, and has been a major non-NATO ally since 2002. After Pakistan\'s participation in the Afghan peace process and the Taliban takeover in Afghanistan in 2021, a sizeable number of US policy makers are revisiting the United States\' relations with Pakistan. At the same time, the strategic convergence of the United States and India has also brought greater pressure on Pakistani diplomacy.', 'source': 'https://en.wikipedia.org/wiki/Pakistan%E

In [3]:
for i, doc in enumerate(docs):
    print(f"\n--- Result {i+1} ---")
    print(f"Content:\n{doc.page_content}...")


--- Result 1 ---
Content:
Pakistan and the United States established relations on 15 August 1947, a day after the independence of Pakistan, when the United States became one of the first nations to recognise the country.
The relationship between the two nations has been described as a "roller coaster" characterised by close coordination and lows marked by deep bilateral estrangement. Despite its troubled history, the Pakistani military once occupied an important place in American geopolitical strategy, and has been a major non-NATO ally since 2002. After Pakistan's participation in the Afghan peace process and the Taliban takeover in Afghanistan in 2021, a sizeable number of US policy makers are revisiting the United States' relations with Pakistan. At the same time, the strategic convergence of the United States and India has also brought greater pressure on Pakistani diplomacy.


== Background ==

During the Cold War (1945–1991), Pakistan allied itself with the Western Bloc led by t

vector store retrieval

In [22]:
from langchain_community.vectorstores import Chroma
from langchain_google_genai.embeddings import GoogleGenerativeAIEmbeddings
from dotenv import load_dotenv
from langchain_core.documents import Document
load_dotenv()

True

In [23]:
documents = [
    Document(page_content="LangChain helps developers build LLM applications easily."),
    Document(page_content="Chroma is a vector database optimized for LLM-based search."),
    Document(page_content="Embeddings convert text into high-dimensional vectors."),
    Document(page_content="OpenAI provides powerful embedding models."),
]

In [40]:
embedding_model=GoogleGenerativeAIEmbeddings(model='models/embedding-001')


vector_store=Chroma.from_documents(
    documents=documents,
    embedding=embedding_model,
    collection_name='my_collection',
    persist_directory="./chroma_db" 
)

In [41]:
retriever1=vector_store.as_retriever(search_kwargs={'k':2})

In [42]:
query='what is chroma used for?'
results=retriever1.invoke(query)
for i,docs in enumerate(results):
    print(f'\n---Result {i+1} ---')
    print(docs.page_content)


---Result 1 ---
Chroma is a vector database optimized for LLM-based search.

---Result 2 ---
Chroma is a vector database optimized for LLM-based search.


MMR

In [53]:
# Sample documents
docs = [
    Document(page_content="LangChain makes it easy to work with LLMs."),
    Document(page_content="LangChain is used to build LLM based applications."),
    Document(page_content="Chroma is used to store and search document embeddings."),
    Document(page_content="Embeddings are vector representations of text."),
    Document(page_content="MMR helps you get diverse results when doing similarity search."),
    Document(page_content="LangChain supports Chroma, FAISS, Pinecone, and more."),
]

In [54]:
from langchain_community.vectorstores import FAISS

vector_store=FAISS.from_documents(
    documents=docs,
    embedding=embedding_model
)


In [62]:
retriever2=vector_store.as_retriever(
    search_type='mmr',
    search_kwargs={'k':2,'lambda_mult':0.5}
)


In [63]:
query='what is mmr?'
results=retriever2.invoke(query)

for i,docs in enumerate(results):
    print(f'\n--- Result {i+1} ---')
    print(docs.page_content)


--- Result 1 ---
MMR helps you get diverse results when doing similarity search.

--- Result 2 ---
Chroma is used to store and search document embeddings.


multi query retriever

In [64]:
from langchain_community.vectorstores import FAISS
from langchain_google_genai import GoogleGenerativeAIEmbeddings,ChatGoogleGenerativeAI
from dotenv import load_dotenv
from langchain_core.documents import Document
from langchain.retrievers.multi_query import MultiQueryRetriever

In [67]:
all_docs = [
    Document(page_content="Regular walking boosts heart health and can reduce symptoms of depression.", metadata={"source": "H1"}),
    Document(page_content="Consuming leafy greens and fruits helps detox the body and improve longevity.", metadata={"source": "H2"}),
    Document(page_content="Deep sleep is crucial for cellular repair and emotional regulation.", metadata={"source": "H3"}),
    Document(page_content="Mindfulness and controlled breathing lower cortisol and improve mental clarity.", metadata={"source": "H4"}),
    Document(page_content="Drinking sufficient water throughout the day helps maintain metabolism and energy.", metadata={"source": "H5"}),
    Document(page_content="The solar energy system in modern homes helps balance electricity demand.", metadata={"source": "I1"}),
    Document(page_content="Python balances readability with power, making it a popular system design language.", metadata={"source": "I2"}),
    Document(page_content="Photosynthesis enables plants to produce energy by converting sunlight.", metadata={"source": "I3"}),
    Document(page_content="The 2022 FIFA World Cup was held in Qatar and drew global energy and excitement.", metadata={"source": "I4"}),
    Document(page_content="Black holes bend spacetime and store immense gravitational energy.", metadata={"source": "I5"}),
]

In [65]:
load_dotenv()

embedding_model=GoogleGenerativeAIEmbeddings(model='models/embedding-001')

model=ChatGoogleGenerativeAI(model='gemini-2.5-pro')



In [68]:
vector_store=FAISS.from_documents(
    documents=all_docs,
    embedding=embedding_model
)


In [74]:
simple_retriever=vector_store.as_retriever(search_type='similarity',search_kwargs={'k':4})

In [75]:
multi_query=MultiQueryRetriever.from_llm(
    llm=model,
    retriever=vector_store.as_retriever(search_type='mmr',search_kwargs={'k':4})
)

In [71]:
query = "How to improve energy levels and maintain balance?"

In [76]:
similarity_results = simple_retriever.invoke(query)
multiquery_results= multi_query.invoke(query)

In [77]:
for i, doc in enumerate(similarity_results):
    print(f"\n--- Result {i+1} ---")
    print(doc.page_content)

print("*"*150)

for i, doc in enumerate(multiquery_results):
    print(f"\n--- Result {i+1} ---")
    print(doc.page_content)


--- Result 1 ---
Drinking sufficient water throughout the day helps maintain metabolism and energy.

--- Result 2 ---
Mindfulness and controlled breathing lower cortisol and improve mental clarity.

--- Result 3 ---
Consuming leafy greens and fruits helps detox the body and improve longevity.

--- Result 4 ---
Deep sleep is crucial for cellular repair and emotional regulation.
******************************************************************************************************************************************************

--- Result 1 ---
Drinking sufficient water throughout the day helps maintain metabolism and energy.

--- Result 2 ---
Mindfulness and controlled breathing lower cortisol and improve mental clarity.

--- Result 3 ---
The solar energy system in modern homes helps balance electricity demand.

--- Result 4 ---
Python balances readability with power, making it a popular system design language.

--- Result 5 ---
Deep sleep is crucial for cellular repair and emotional r

contextual compressing retrieval

In [81]:
from langchain_community.vectorstores import FAISS
from langchain_google_genai import ChatGoogleGenerativeAI,GoogleGenerativeAIEmbeddings
from langchain.retrievers.contextual_compression import ContextualCompressionRetriever
from langchain.retrievers.document_compressors import LLMChainExtractor
from langchain_core.documents import Document

In [82]:
# Recreate the document objects from the previous data
docs = [
    Document(page_content=(
        """The Grand Canyon is one of the most visited natural wonders in the world.
        Photosynthesis is the process by which green plants convert sunlight into energy.
        Millions of tourists travel to see it every year. The rocks date back millions of years."""
    ), metadata={"source": "Doc1"}),

    Document(page_content=(
        """In medieval Europe, castles were built primarily for defense.
        The chlorophyll in plant cells captures sunlight during photosynthesis.
        Knights wore armor made of metal. Siege weapons were often used to breach castle walls."""
    ), metadata={"source": "Doc2"}),

    Document(page_content=(
        """Basketball was invented by Dr. James Naismith in the late 19th century.
        It was originally played with a soccer ball and peach baskets. NBA is now a global league."""
    ), metadata={"source": "Doc3"}),

    Document(page_content=(
        """The history of cinema began in the late 1800s. Silent films were the earliest form.
        Thomas Edison was among the pioneers. Photosynthesis does not occur in animal cells.
        Modern filmmaking involves complex CGI and sound design."""
    ), metadata={"source": "Doc4"})
]

In [83]:
embedding_model = GoogleGenerativeAIEmbeddings(model='models/embedding-001')
vectorstore = FAISS.from_documents(docs, embedding_model)
base_retriever = vectorstore.as_retriever(search_kwargs={"k": 5})

In [84]:
llm = ChatGoogleGenerativeAI(model='gemini-2.5-pro')
compressor = LLMChainExtractor.from_llm(llm)

In [85]:
compression_retriever = ContextualCompressionRetriever(
    base_retriever=base_retriever,
    base_compressor=compressor
)

In [86]:
query = "What is photosynthesis?"
compressed_results = compression_retriever.invoke(query)

In [87]:
for i, doc in enumerate(compressed_results):
    print(f"\n--- Result {i+1} ---")
    print(doc.page_content)


--- Result 1 ---
Photosynthesis does not occur in animal cells.

--- Result 2 ---
Photosynthesis is the process by which green plants convert sunlight into energy.

--- Result 3 ---
The chlorophyll in plant cells captures sunlight during photosynthesis.
